In [2]:
pip install google-generativeai python-pptx Pillow requests python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import google.generativeai as genai
from dotenv import load_dotenv
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor
from PIL import Image
import requests
import io
import json

load_dotenv()

c:\Users\Pranab Samanta\Desktop\presentation-generator\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [ ]:
import time


class PPTGenerator:
  def __init__(self, api_key = None):
    self.api_key = api_key
    if not self.api_key:
      raise ValueError('Gemini API key is not available')

    genai.configure(api_key = self.api_key)
    self.model = genai.GenerativeModel('gemini-2.5-pro')
    self.model_version = genai.GenerativeModel('gemini-2.5-pro')

    self.presentation = Presentation()
    
   
    self.last_api_call = 0
    self.min_delay = 30 
    self.api_call_count = 0

  def _wait_for_rate_limit(self):
    """Enforce rate limiting between API calls"""
    current_time = time.time()
    time_since_last_call = current_time - self.last_api_call
    
    if time_since_last_call < self.min_delay:
      wait_time = self.min_delay - time_since_last_call
      print(f" Rate limiting: Waiting {wait_time:.1f}s before next API call...")
      time.sleep(wait_time)
    
    self.last_api_call = time.time()
    self.api_call_count += 1
    
  # This function retries API calls with exponential backoff for rate limit and 503 errors.
  def _retry_api_call(self, func, *args, max_retries=3, **kwargs):
    """Retry API calls with exponential backoff"""
    for attempt in range(max_retries):
      try:
        self._wait_for_rate_limit()
        return func(*args, **kwargs)
      except Exception as e:
        error_msg = str(e)
        
        # Check for rate limit error
        if "429" in error_msg or "quota" in error_msg.lower():
          if attempt < max_retries - 1:
            wait_time = 45 * (attempt + 1)  # 45s, 90s, 135s
            print(f"Rate limit hit. Waiting {wait_time}s before retry {attempt + 2}/{max_retries}...")
            time.sleep(wait_time)
            continue
          else:
            print(f"Max retries reached. Skipping this API call.")
            return None
        
        # Check for 503 error
        elif "503" in error_msg:
          if attempt < max_retries - 1:
            wait_time = 5 * (attempt + 1)
            print(f"Service unavailable (503). Retrying in {wait_time}s...")
            time.sleep(wait_time)
            continue
          else:
            print(f"Service unavailable after {max_retries} attempts.")
            return None
        
        # Other errors
        else:
          print(f"Error: {e}")
          return None
    
    return None
  
  # This function generates content outlines using Gemini API with retry logic.

  def generate_content_outlines(self, topic, num_slides=5):
    '''Generate content outline using Gemini with retry logic'''
    prompt = f"""
    Create a detailed outline for a PowerPoint presentation on "{topic}" with {num_slides} slides.
    Return the response as a JSON array with the following structure:
    [
      {{
        "title": "Slide Title",
        "content": "Main content points as bullet points",
        "slide_type": "title | content | image | conclusion"
      }}
    ]
    Make sure the content is engaging, informative, and well-structured.
    The response must be a valid JSON array.
    """

    def generate():
      response = self.model.generate_content(prompt)
      return response.text.strip()

    try:
        content = self._retry_api_call(generate)
        
        if content is None:
          raise ValueError("Failed to generate content after retries")

        # clean code fences if present
        if "```json" in content:
            content = content.split("```json")[1].split("```")[0].strip()
        elif "```" in content:
            content = content.split("```")[1].strip()

        # force fallback if not JSON
        if not content.startswith('['):
            print("Gemini did not return JSON. Using fallback content.")
            return self.get_fallback_content(topic)

        return json.loads(content)

    except Exception as e:
        print(f"Error generating content: {e}")
        return self.get_fallback_content(topic)
      
      
      # Fallback content if API fails or returns invalid data

  def get_fallback_content(self, topic):
    """Return generic fallback content"""
    return [
        {"title": f"{topic}", "content": "Introduction and Overview", "slide_type": "title"},
        {"title": "Background", "content": "Key background information", "slide_type": "content"},
        {"title": "Main Points", "content": "Important details and examples", "slide_type": "content"},
        {"title": "Analysis", "content": "Critical analysis and insights", "slide_type": "content"},
        {"title": "Future Outlook", "content": "Trends and predictions", "slide_type": "content"},
        {"title": "Conclusion", "content": "Summary and takeaways", "slide_type": "conclusion"}
    ]

  def generative_image_description(self, slide_content):
    """Generate image description with fallback for common topics"""
   
    content_lower = str(slide_content).lower()
    
    
    keyword_map = {
      'introduction': 'professional business presentation',
      'conclusion': 'success celebration team',
      'future': 'futuristic technology innovation',
      'challenge': 'business strategy planning',
      'benefit': 'growth success chart',
      'process': 'workflow diagram business',
      'team': 'diverse team collaboration',
      'ai': 'artificial intelligence technology',
      'movie': 'film production cinema',
      'production': 'movie set film making',
      'technology': 'modern technology innovation'
    }
    
    # Check for keywords first
    for keyword, image_desc in keyword_map.items():
      if keyword in content_lower:
        print(f"Using cached image description: {image_desc}")
        return image_desc
    
    # If no keyword match, use API with retry
    prompt = f"""
    Based on this slide content, suggest a relevant image description that would enhance the presentation
    {slide_content}
    Return only a brief, descriptive phrase suitable for image search (max 7 words)
    """
    
    def generate():
      response = self.model.generate_content(prompt)
      return response.text.strip()
    
    try:
      result = self._retry_api_call(generate)
      return result if result else 'professional presentation visual'
    except Exception as e:
      print(f"Error generating image description: {e}")
      return 'professional presentation visual'
    
    # This function downloads images from Pexels based on a query. 

  def download_images(self, query, save_path ='temp_image.jpg'):
    try:
      url ='https://api.pexels.com/v1/search'
      header ={
          'Authorization': PEXEL_API_KEY
      }

      params ={
          'query': query,
          'per_page': 1,
          'orientation': 'landscape'
      }

      response = requests.get(url, headers = header, params = params)
      response.raise_for_status()

      data = response.json()
      if not data.get('photos'):
        raise ValueError('No photo found')

      image_url = data['photos'][0]['src']['original']
      image_response = requests.get(image_url)
      image_response.raise_for_status()

      # save image
      with open(save_path, 'wb') as f:
        f.write(image_response.content)

      return save_path

    except Exception as e:
      print(f"Error downloading image: {e}")
      return None
    
    
    # This function creates a title slide with given title and subtitle.
    

  def create_title_slide(self, title, subtitle):
    slide_layout = self.presentation.slide_layouts[0]
    slide = self.presentation.slides.add_slide(slide_layout)

    title_shape = slide.shapes.title
    title_shape.text = title

    title_shape.text_frame.paragraphs[0].font.size = Pt(30)
    title_shape.text_frame.paragraphs[0].font.bold = True
    title_shape.text_frame.paragraphs[0].font.color.rgb = RGBColor(0, 0, 0)
    title_shape.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

    if subtitle:
      subtitle_shape = slide.placeholders[1]
      subtitle_shape.text = subtitle
      subtitle_shape.text_frame.paragraphs[0].font.size = Pt(20)
      subtitle_shape.text_frame.paragraphs[0].font.color.rgb = RGBColor(0, 0, 0)
      subtitle_shape.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
      
      
  # This function creates a content slide with optional image inclusion.


  def create_content_slide(self, title, content, include_image=False):
    slide_layout = self.presentation.slide_layouts[1]
    slide = self.presentation.slides.add_slide(slide_layout)

    title_shape = slide.shapes.title
    title_shape.text = title
    title_shape.text_frame.paragraphs[0].font.size = Pt(30)
    title_shape.text_frame.paragraphs[0].font.bold = True

    # Handle both string and list content 
    content_shape = slide.placeholders[1]
    text_frame = content_shape.text_frame
    text_frame.clear()
    
    if isinstance(content, list):
        content_text = '\n'.join(str(item) for item in content)
    else:
        content_text = str(content)
    
    p = text_frame.paragraphs[0]
    p.text = content_text
    p.font.size = Pt(18)
    p.font.color.rgb = RGBColor(0, 0, 0)

    if include_image:
        try:
            image_desc = self.generative_image_description(content_text)
            if image_desc:
              image_path = self.download_images(image_desc)
              if image_path and os.path.exists(image_path):
                  slide.shapes.add_picture(image_path, Inches(6), Inches(2), height=Inches(4))
                  os.remove(image_path)
        except Exception as e:
            print(f"Error adding image to slide: {e}")

    return slide
  
  
  

  def create_image_slide(self, title, content, image_query):
    slide_layout = self.presentation.slide_layouts[8]
    slide = self.presentation.slides.add_slide(slide_layout)

    # This layout has no title placeholder, so we add a textbox for title
    title_box = slide.shapes.add_textbox(Inches(0.5), Inches(0.5), Inches(9), Inches(0.8))
    title_frame = title_box.text_frame
    title_frame.text = title
    title_frame.paragraphs[0].font.size = Pt(30)
    title_frame.paragraphs[0].font.bold = True
    title_frame.paragraphs[0].font.color.rgb = RGBColor(0, 0, 0)
    title_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

    # Add content textbox below title if content is provided
    content_box = slide.shapes.add_textbox(Inches(1), Inches(1.2), Inches(8), Inches(2.5))
   
    content_frame = content_box.text_frame
    
    if isinstance(content, list):
        content_text = '\n'.join(str(item) for item in content)
    else:
        content_text = str(content)
    
    content_frame.text = content_text

    for paragraph in content_frame.paragraphs:
      paragraph.font.size = Pt(15)
      paragraph.font.color.rgb = RGBColor(51, 51, 51)

    try:
      image_path = self.download_images(image_query)
      if image_path and os.path.exists(image_path):
        slide.shapes.add_picture(image_path, Inches(3.25), Inches(4), width=Inches(3.5), height=Inches(2.5))
        os.remove(image_path)
    except Exception as e:
      print(f"Error adding image to slide: {e}")

    return slide
  
  #This is the main funtion to generate ppt file and save it. 

  def generate_ppt(self, topic, num_slides=6, output_file='ppptt.pptx'):
    print(f"Starting PPT generation for: {topic}")
    print(f"Total slides: {num_slides}")
    print(f"Estimated time: ~{num_slides * 30 // 60} minutes (due to rate limits)\n")
    
    content_outlines = self.generate_content_outlines(topic, num_slides)

    for i, slide_data in enumerate(content_outlines):
      title = slide_data.get('title', f"Slide {i+1}")
      content = slide_data.get('content', "")
      slide_type = slide_data.get('slide_type', 'content')

      print(f"\n Generating slide {i+1}/{len(content_outlines)}: {title}")

      if i == 0 or slide_type == 'title':
          self.create_title_slide(title, 'Created by Pranab')
      elif slide_type == 'content':
          self.create_content_slide(title, content, include_image=True)
      elif slide_type == 'image':
          content_str = '\n'.join(content) if isinstance(content, list) else str(content)
          img_query = self.generative_image_description(content_str)
          self.create_image_slide(title, content, img_query)
      else:
          self.create_content_slide(title, content, include_image=False)

    self.presentation.save(output_file)
    print(f"\n PPT generated successfully at {output_file}")
    print(f" Total API calls made: {self.api_call_count}")

print("Generation Done")

In [ ]:
api_key ='AIzaSyBhsFdiDUPWokiBjDomkzc'
PEXEL_API_KEY ='wftdJGjbtKP4Nm4zhOYhGk1YetBhwtEIA6XTUu'

In [ ]:
from typing import ValuesView
try:
  generator = PPTGenerator(api_key= api_key)
except ValueError as e:
  print(f"Error creating PPT generator: {e}")


In [ ]:
topic = 'AI in Video Editing Industry'
num_slides = 7

print(f"Topic: {topic}")
print(f"Number of slides: {num_slides}")

try:
  output_file = 'presentation.pptx'
  generator.generate_ppt(topic, num_slides, "video_editing.pptx")
except Exception as e:
  print(f"Error generating PPT: {e}")